In [ ]:
#hide
#default_exp fname_processing
from nbdev.showdoc import show_doc

# Filename Processing
> Classify audio files according to their noise level and audio type.

In [ ]:
#export
import pathlib
import os
import tensorflow as tf
import numpy as np
import pickle
import string

### Functions for filtering data

In [ ]:
#exports
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    label_tensor = tf.strings.substr(parts[-1], pos=9, len=1)
    label = label_tensor.numpy().decode('utf-8')
    return label

def check_type(fname, dataset_type):
    label = get_label(fname)
    if dataset_type == 'digits' and label.isdigit():
        return True
    elif dataset_type == 'letters' and not(label.isdigit()):
        return True
    return False

def get_fnames(data_dir):
    return tf.io.gfile.glob(str(data_dir)+"*/*.wav")

def filter_fnames(fnames):
    return [fname for fname in fnames if check_type(fname, dataset_type)]

def show_data_count(filenames, label_strings):
    for i in range(len(label_strings)):
        num_examples = len([fname for fname in filenames if get_label(fname)==label_strings[i]])
        print(f"""# examples for "{label_strings[i]}": {num_examples}""")
    
    num_samples = len(filenames)
    print('# total examples:', num_samples)
    print()

### Functions for classifying noise levels

In [ ]:
#exports
noise_levels = ['IDL', '35D', '35U', '55D', '55U']

def get_noise_level(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    level_tensor = tf.strings.substr(parts[-1], pos=4, len=3)
    level = level_tensor.numpy().decode('utf-8')
    return level

def classify_noise_levels(fnames):
    noise_levels = ['IDL', '35D', '35U', '55D', '55U']
    noise_levels_dict = {}
    # initialize empty lists
    for level in noise_levels:
        noise_levels_dict[level] = []
    # put each filename into a category
    for fname in fnames:
        noise_level = get_noise_level(fname)
        noise_levels_dict[noise_level].append(fname)
    return noise_levels_dict
    
    

### Functions for loading classified filenames

In [ ]:
#exports
def load_fnames(fname_path):
    with open(fname_path, 'rb') as filehandle:
        return pickle.load(filehandle)

## Digits Data

In [ ]:
dataset_type = 'digits'
data_dir = pathlib.Path('data/avicar_some'+dataset_type)
label_strings = np.array([str(num) for num in range(0,10)])

In [ ]:
filenames = get_fnames(data_dir)
print("Example filename:", filenames[99])
print()

# Filter out non-digit files
filenames = filter_fnames(filenames)

# Count # of examples for each label
show_data_count(filenames, label_strings)

Example filename: data/avicar_somedigits/AM3_35U_D0_C1_M3.wav

# examples for "0": 772
# examples for "1": 771
# examples for "2": 771
# examples for "3": 771
# examples for "4": 770
# examples for "5": 771
# examples for "6": 771
# examples for "7": 770
# examples for "8": 769
# examples for "9": 772
# total examples: 7708



In [ ]:
# Classify
noise_level_dict = classify_noise_levels(filenames)

In [ ]:
# Put filenames into csv
noise_level_dir = 'noise_levels'
digit_dir = 'digit_noise_levels'
csv_path = os.path.join(noise_level_dir, digit_dir)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
    for level in noise_level_dict:
        name = level + '.data'
        file_path = os.path.join(csv_path, name)
        with open(file_path, 'wb') as filehandle:
            pickle.dump(noise_level_dict[level], filehandle)

## Letters Data

In [ ]:
dataset_type = 'letters'
data_dir = pathlib.Path('data/avicar_some'+dataset_type)
label_strings = np.array(list(string.ascii_uppercase))

In [ ]:
filenames = get_fnames(data_dir)
print("Example filename:", filenames[99])
print()

# Filter out non-digit files
filenames = filter_fnames(filenames)

# Count # of examples for each label
show_data_count(filenames, label_strings)

Example filename: data/avicar_someletters/AM5_35U_LN_C1_M3.wav

# examples for "A": 543
# examples for "B": 543
# examples for "C": 544
# examples for "D": 544
# examples for "E": 544
# examples for "F": 544
# examples for "G": 543
# examples for "H": 543
# examples for "I": 544
# examples for "J": 544
# examples for "K": 544
# examples for "L": 544
# examples for "M": 542
# examples for "N": 543
# examples for "O": 543
# examples for "P": 544
# examples for "Q": 543
# examples for "R": 544
# examples for "S": 543
# examples for "T": 544
# examples for "U": 544
# examples for "V": 543
# examples for "W": 544
# examples for "X": 544
# examples for "Y": 543
# examples for "Z": 544
# total examples: 14132



In [ ]:
# Classify
noise_level_dict = classify_noise_levels(filenames)

In [ ]:
# Put filenames into csv
noise_level_dir = 'noise_levels'
digit_dir = 'letter_noise_levels'
csv_path = os.path.join(noise_level_dir, digit_dir)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
    for level in noise_level_dict:
        name = level + '.data'
        file_path = os.path.join(csv_path, name)
        with open(file_path, 'wb') as filehandle:
            pickle.dump(noise_level_dict[level], filehandle)

## Mixed Data

In [ ]:
dataset_type = 'mixed'
data_dir = pathlib.Path('data/avicar_'+dataset_type)
label_strings = label_strings = np.array([str(num) for num in range(0,10)] + list(string.ascii_uppercase))

In [ ]:
filenames = get_fnames(data_dir)
print("Example filename:", filenames[99])
print()

# Filter out DD, DT, DZ
filenames = [fname for fname in filenames if fname[26:28] not in ['DD', 'DT', 'DZ']]

# Count # of examples for each label
show_data_count(filenames, label_strings)

Example filename: data/avicar_mixed/IF3_35D_LS_C1_M3.wav

# examples for "0": 772
# examples for "1": 771
# examples for "2": 771
# examples for "3": 771
# examples for "4": 770
# examples for "5": 771
# examples for "6": 771
# examples for "7": 770
# examples for "8": 769
# examples for "9": 772
# examples for "A": 543
# examples for "B": 543
# examples for "C": 544
# examples for "D": 544
# examples for "E": 544
# examples for "F": 544
# examples for "G": 543
# examples for "H": 543
# examples for "I": 544
# examples for "J": 544
# examples for "K": 544
# examples for "L": 544
# examples for "M": 542
# examples for "N": 543
# examples for "O": 543
# examples for "P": 544
# examples for "Q": 543
# examples for "R": 544
# examples for "S": 543
# examples for "T": 544
# examples for "U": 544
# examples for "V": 543
# examples for "W": 544
# examples for "X": 544
# examples for "Y": 543
# examples for "Z": 544
# total examples: 21840



In [ ]:
# Classify
noise_level_dict = classify_noise_levels(filenames)

In [ ]:
# Put filenames into csv
noise_level_dir = 'noise_levels'
digit_dir = 'mixed_noise_levels'
csv_path = os.path.join(noise_level_dir, digit_dir)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
    for level in noise_level_dict:
        name = level + '.data'
        file_path = os.path.join(csv_path, name)
        with open(file_path, 'wb') as filehandle:
            pickle.dump(noise_level_dict[level], filehandle)